1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ load the `txt` file of the previous point and convert it to a `csv` file by hand.

In [16]:
list=[1,2,3,4,5]
file_name="data_int.txt"
file = open(file_name, 'w')
file.write(str(list))
file.close() 
!cat data_int.txt

[1, 2, 3, 4, 5]

2\. **JSON files**

Load the file `user_data.json`, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [12]:
import json
import pandas as pd
data=json.load(open('/home/kumarvinay/Testfolder/ScientificComputingWithPython2023/user_data.json'))
output_data = [x for x in data if x['CreditCardType'] == 'American Express']
output_json=json.dumps(output_data)
df = pd.read_json(output_json)
df.to_csv("output.csv", index=False)

3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [18]:
import pandas as pd
file_name= "mushrooms_categorized.csv"
data = pd.read_csv(file_name)
data.head()


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


4\. **Reading a database**

Get the database `sakila.db` from the lecture `06_dataio.ipynb`, and import the table `actors` as a Pandas dataframe. Using the dataframe, count how many actors have a first name that begins with `A`.

*Hint:* use the Series `.str` method to apply the Python string methods to the elements of a Series, see [documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.html).

In [50]:
import sqlite3 as sql

# create a connection to the database and a cursor to execute queries
conn = sql.connect('data/sakila.db')
cur = conn.cursor()
query = "SELECT * FROM actor "
results = cur.execute(query).fetchall()

# create a DataFrame from the DB data
df = pd.DataFrame(results)
#print(df[df.columns[pd.Series(df.columns).str.startswith('A')]])
Pdf.filter(regex='A*')
cur.close()
conn.close()
df

,0,1,2,3
0,1,PENELOPE,GUINESS,2019-02-16 18:17:33
1,2,NICK,WAHLBERG,2019-02-16 18:17:33
2,3,ED,CHASE,2019-02-16 18:17:33
3,4,JENNIFER,DAVIS,2019-02-16 18:17:33
4,5,JOHNNY,LOLLOBRIGIDA,2019-02-16 18:17:33
...,...,...,...,...
195,196,BELA,WALKEN,2019-02-16 18:17:33
196,197,REESE,WEST,2019-02-16 18:17:33
197,198,MARY,KEITEL,2019-02-16 18:17:33
198,199,JULIA,FAWCETT,2019-02-16 18:17:33


5\. **Reading the credit card numbers**

Get the binary file named `credit_card.dat` from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

6\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [ ]:
from IPython.display import Image
Image("images/data_format.png")

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, "pack" the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_dataio.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?